## Inversion for a coupled acoustic-gravity system

Before running `pip install tinyda ray' 

### Problem setup

Let $\varphi$ be the potential for the flow velocity, it is solution to 
\begin{align}\label{eq:forwardmodel}
    \frac{\partial^2 \varphi}{\partial t^2}
    - c^2 \Delta \varphi 
    + g \frac{\partial \varphi}{\partial z}
    = 0,  \quad \text{ in } \Omega,
    \\
    \frac{\partial^2 \varphi}{\partial t^2}
    + g \frac{\partial \varphi}{\partial z} 
    = 0, \quad \text{ on } \Gamma_s,
    \\
    \nabla \varphi \cdot {\bf n}
    = u_b, \quad \text{ on } \Gamma_b,
\end{align}
where $c$ is the (constant) sound speed, $g$ is the gravity acceleration, ${\bf n}$ is the outwards unit vector normal to the seabed $\Gamma_b$ and $u_b$ is the seabed velocity.
The fluid displacement $u$ is related to the potential by $u = \nabla \varphi$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, norm, rv_continuous, wasserstein_distance
from scipy.spatial.distance import cdist
from scipy.spatial import distance_matrix

import tinyDA as tda
import umbridge

### Initialising UM-Bridge
Before running the following cell, the model Docker image must be started from a system terminal using

`docker run -it -p 4242:4242 acousticgravity`

The Dockerfile can be found in the acousticgracity branch on UM-Bridge benchmarks. If not just running on a local machine replace `localhost` with the server.

In [ ]:
# connect to the UM-Bridge model.
umbridge_model = umbridge.HTTPModel('http://localhost:4242', "forward")

# wrap the UM-Bridge model in the tinyDA UM-Bridge interface.
my_model = tda.UmBridgeModel(umbridge_model)

In [ ]:
nx = umbridge_model.get_input_sizes()[0] #the input is the value of f(x) at each grid point
ny = umbridge_model.get_output_sizes()[0] #the outout is a time series of the pressure

In [ ]:
length_scale = 5.0
eps = 1e-6
decay_rate = 10

x = np.linspace(0, nx - 1, nx).reshape(-1, 1)

def rbf_covariance(grid, length_scale, variance=1.0):
    dists = cdist(grid, grid, 'euclidean')
    return variance * np.exp(-0.5 * (dists / length_scale) ** 2)

cov = rbf_covariance(x, length_scale)
cov += eps * np.eye(nx)
mean = np.zeros(nx)
my_prior = multivariate_normal(mean=mean, cov=cov)

def smooth_cut(x, x0, width, k):
    step1 = 1 / (1 + np.exp(-k * (x - (x0 - width / 2))))
    step2 = 1 / (1 + np.exp(-k * (x - (x0 + width / 2))))
    return step1 - step2

cut = smooth_cut(x.flatten(), nx / 2, nx * (1 - 14 / nx), decay_rate)

sample = my_prior.rvs()

def restriction(sampin):
    field_cut = sampin * cut
    return (field_cut > eps).astype(int)

sample = restriction(sample)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))
ax.set_xticks([i for i in range(nx)])
ax.grid()
ax.plot(sample,label='sample')
ax.plot(cut,label='smooth cut')
ax.plot(sample * cut, label='cut sample')
ax.legend()
ax.set_title("Binary Level Set Sample from Gaussian Random Field")
plt.show()

### Problem setup

In [ ]:
# generate synthetic data for input 0.5
# Input is a 15km wide fault centered around x
# Domain size is 150km
exact = np.zeros(nx)
exact[20:25]=1
d_true = my_model(exact)

# add some noise to the model output
sigma_noise = 0.001
d = d_true + np.random.normal(loc=0, scale=sigma_noise, size=ny)

In [ ]:
fig, ax = plt.subplots(figsize=(12,2))
ax.set_title('Input')
x=np.linspace(0,nx, nx)
t=np.linspace(0,10, ny)
ax.plot(t,d_true, label='Noise-free output')
ax.legend()

We plot only model output as the input is a simple 1D location so far

In [ ]:
# plot the model output
fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(12,4))

# plot the time series of pressure
t = np.linspace(0,10,ny)
ax[0].set_title('Pressure')
ax[0].plot(t,d_true, label='Noise-free input')
ax[0].plot(t,d, label='Noisy input')
ax[0].legend()

ax[1].set_title('Pressure')
samples = []
inputs=[]
# Plot 10 random samples
for x in range(15):
    log = my_prior.rvs()
    log = restriction(log)
    d_sample = my_model(log)
    ax[1].plot(t, d_sample, label='samples')
    ax[2].plot(range(len(log)), log, label='input')
    samples.append(d_sample)
    inputs.append(log)
plt.show()

## Set up log-likelihood

In [ ]:
class wasserstein_loglike():
    def __init__(self, ground_truth):
        self.ground_truth = ground_truth
    
    def loglike(self,x):
        return -100 * wasserstein_distance(x,self.ground_truth)

In [ ]:
# # set the covariance of the likelihood.
# sigma = 0.125
# cov_likelihood = sigma**2*np.eye(d.shape[0])
# my_loglike = tda.GaussianLogLike(d, cov_likelihood)
my_loglike = wasserstein_loglike(d_true)

# initialise the Posterior
my_posterior = tda.Posterior(my_prior, my_loglike, my_model)

In [ ]:
for d_sample in samples:
    print(np.exp(my_loglike.loglike(d_sample)))

In [ ]:
# plot the model output
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(12,4))

ax[1].set_title('Pressure')
# Plot 10 random samples
for i in range(5):
    #log = my_prior.rvs()
    #d_sample = my_model(log)
    log = inputs[i]
    print(log)
    ax[0].plot(log)
    ax[0].plot(28,0.9, 'x', color='red')
    ax[1].plot(t, samples[i], label=f'samples {i}')
    #samples.append(d_sample)
ax[1].legend()
plt.show()

### Set up the proposal

In [ ]:
# Custom Crank-Nicolson with cutoff and levelset
class CrankNicolsonWithProjection(tda.CrankNicolson):
    def __init__(self, scaling, adaptive):
        super().__init__(scaling=scaling, adaptive=adaptive)

    def make_proposal(self, link):
        # make a pCN proposal.
        return np.sqrt(
            1 - self.scaling**2
        ) * link.parameters + self.scaling * restriction(np.random.multivariate_normal(
            self._mean, self.C
        ))

# preconditioned Crank-Nicolson
pcn_scaling = 1.0
pcn_adaptive = False
my_proposal = CrankNicolsonWithProjection(scaling=pcn_scaling, adaptive=pcn_adaptive)

# # random walk Metropolis
#rwmh_cov = np.eye(nx)
#rmwh_scaling = 0.1
#rwmh_adaptive = False
#my_proposal = tda.GaussianRandomWalk(C=rwmh_cov, scaling=rmwh_scaling, adaptive=rwmh_adaptive)

### Sample

In [ ]:
# For testing purposes, iteration number is small for the given problem; Choose a larger number for real applications.
my_chains = tda.sample(my_posterior, my_proposal, iterations=10, n_chains=2, force_sequential=True)

### Get some diagnostics

In [ ]:
import arviz as az

In [ ]:
# convert the tinyDA chains to an ArViz InferenceData object.
idata = tda.to_inference_data(my_chains, burnin=1000)

In [ ]:
# display posterior summary statistics.
info = az.summary(idata)
info

In [ ]:
# plot posterior kernel densities and traces.
az.plot_trace(idata, figsize=(18, 18))
plt.show()

In [ ]:
# extract the parameters from the chains.
parameters = [link.parameters for link in my_chains['chain_0'][2001:] + my_chains['chain_1'][2001:]]

In [ ]:
# plot some posterior draws of the model input and output.
n_samples = 1000
ids = np.random.randint(0, len(parameters), n_samples)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,4))

# plot the output pressure
ax.set_title('Pressure')
ax.plot(np.linspace(0,1,ny), d)
for i in ids:
    ax.plot(np.linspace(0,1,ny), my_model(parameters[i]), c='k', alpha=0.01)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,2))
ax.set_title('Input')
x=np.linspace(0,nx, nx)
t=np.linspace(0,10, ny)
#ax.plot(x,exact, label='Noise-free input')
ax.plot(t,d_true, label='Noise-free output')
ax.legend()

exact = np.zeros(nx)
#exact[int(nx/2 - 6):int(nx/2 + 5)]=1
exact[20:25]=1

fig,ax=plt.subplots(figsize=(10,3))
ax.set_xticks([i for i in range(44)])
ax.grid()

ax.plot(exact)

x = range(len(info))
plt.figure(figsize=(8, 6))
plt.plot(x, info['mean'], 'o')  # x = index, y = mean
plt.xticks(x, info.index, rotation=90)  # Show variable names on x-axis
plt.ylabel('Mean')
plt.xlabel('Variable')
plt.title('Posterior Means')
plt.grid(True)
plt.tight_layout()
plt.show()

# Sequential MC

I've tried out this particle filter on an example they give. Laura, does the package contain anything useful?

In [ ]:
from pfilter import ParticleFilter, independent_sample, squared_error
from scipy.stats import norm, gamma, uniform 
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
def apply_filter(pf, ys, inputs=None):
    """Apply filter pf to a series of observations (time_steps, h)  and return a dictionary:    
        particles: an array of particles (time_steps, n, d)
        weights: an array of weights (time_steps,)        
    """

    states = []
    pf.init_filter()  # reset
    for i,y in enumerate(ys):
        if inputs is None:
            pf.update(y)
        else:
            pf.update(y, **inputs[i])
            
        states.append([pf.transformed_particles, np.array(pf.weights)])
    return {
        name: np.array([s[i] for s in states])
        for i, name in enumerate(["particles", "weights"])
    }

In [ ]:
def plot_particles(x, y, yn, states):
    """Plot a 1D tracking result as a line graph with overlaid
    scatterplot of particles. Particles are sized according to
    normalised weight at each step.
    
        x: time values
        y: original (uncorrupted) values
        yn: noisy (observed) values
        states: dictionary return from apply_pfilter        
    """
    fig, ax = plt.subplots()
    ax.plot(x, y, label='True', lw=1)
    ax.plot(x, yn, label='Noisy', lw=2)
    
    particles = states["particles"]
    ws = states["weights"]    
    means = np.sum(particles[:,:,0] * ws, axis=1)
    
    dev = (means - (particles[:,:,0]).T).T**2
    var = np.sum(ws * dev, axis=1)  / 1-np.sum(ws**2)  # unbiased variance
    stds = np.sqrt(var)
    
    ax.plot(x, means, 'C4', label='Mean est.', lw=4)
    ax.fill_between(x, means-stds, means+stds, color='C4', alpha=0.5, label='Std.')
    ax.scatter(np.tile(x, (len(particles[0]),1)).ravel(), particles[:,:,0].T, s=ws*1000/np.sqrt(len(ws)),
                alpha=0.15, label='Particles')
    ax.set_xlabel("Time")
    ax.set_ylabel("Observed")
    ax.legend()
    
def filter_plot(x, y, yn, pf, inputs=None):
    """Apply a filter to yn, and plot the results using plot_particles()"""
    states = apply_filter(pf, yn, inputs)
    plot_particles(x, y, yn, states)

In [ ]:
# Noisy sine wave data
x = np.linspace(0, 100, 100)
y = np.cos(x/4.0) + x * 0.05
yn = y + np.random.normal(0,0.5,x.shape)


fig, ax = plt.subplots()
ax.plot(x, y, label='True', lw=1)
ax.plot(x, yn, label='Noisy', lw=1)


In [ ]:
# No dynamics
# just diffusion on x
prior_fn = lambda n: np.random.normal(0,1,(n,1))
dt = 0.05
noise = 0.15
sigma = 1.5

pf = ParticleFilter(prior_fn = prior_fn, 
                    observe_fn = lambda x:  x,  
                    dynamics_fn=lambda x:   x ,
                    n_particles=250, 
                    noise_fn = lambda x: x + np.random.normal(0, noise, x.shape),
                    weight_fn = lambda x,y : squared_error(x, y, sigma=sigma),
                    resample_proportion=0.01)

filter_plot(x, y, yn, pf)

# Level set priors

Some samples from 2D and 1D level sets. Comparing cut of a 2D level set and 1D.

In [ ]:
%matplotlib inline
from FyeldGenerator import generate_field
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
import numpy as np
%pylab inline

In [ ]:
# Helper that generates power-law power spectrum
def Pkgen(n):
    def Pk(k):
        return np.power(k, -n)
    return Pk

# Draw samples from a normal distribution
def distrib(shape):
    a = np.random.normal(loc=0, scale=1, size=shape)
    b = np.random.normal(loc=0, scale=1, size=shape)
    return a + 1j * b

In [ ]:
shape = (512, 512)
field1 = generate_field(distrib, Pkgen(2), shape)
field2 = generate_field(distrib, Pkgen(4), shape)
field3 = generate_field(distrib, Pkgen(6), shape)
field4 = generate_field(distrib, Pkgen(2), shape)
field5 = generate_field(distrib, Pkgen(4), shape)
field6 = generate_field(distrib, Pkgen(6), shape)

fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(12,8))
ax[0,0].imshow(field1, cmap="jet")
ax[0,1].imshow(field2, cmap="jet")
ax[0,2].imshow(field3, cmap="jet")
ax[1,0].imshow(field1<0, cmap="jet")
ax[1,1].imshow(field2<0, cmap="jet")
ax[1,2].imshow(field3<0, cmap="jet")
plt.show()

fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(12,8))
ax[0,0].imshow(field4, cmap="jet")
ax[0,1].imshow(field5, cmap="jet")
ax[0,2].imshow(field6, cmap="jet")
ax[1,0].imshow(field4<0, cmap="jet")
ax[1,1].imshow(field5<0, cmap="jet")
ax[1,2].imshow(field6<0, cmap="jet")
plt.show()

In [ ]:
# Cross section of a 2D randomfield
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(12,4))

ax[0].plot(field1[0,:]<0)
ax[1].plot(field2[0,:]<0)
ax[2].plot(field3[0,:]<0)
plt.show()

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(12,4))

ax[0].plot(field4[0,:]<0)
ax[1].plot(field5[0,:]<0)
ax[2].plot(field6[0,:]<0)
plt.show()

In [ ]:
shape = (512, 1)
field1 = generate_field(distrib, Pkgen(2), shape)
field2 = generate_field(distrib, Pkgen(4), shape)
field3 = generate_field(distrib, Pkgen(6), shape)
field4 = generate_field(distrib, Pkgen(2), shape)
field5 = generate_field(distrib, Pkgen(4), shape)
field6 = generate_field(distrib, Pkgen(6), shape)

# 1D levelset
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(12,8))

ax[0,0].plot(field1)
ax[0,1].plot(field2)
ax[0,2].plot(field3)
ax[1,0].plot(field1<0)
ax[1,1].plot(field2<0)
ax[1,2].plot(field3<0)
plt.show()

fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(12,8))

ax[0,0].plot(field4)
ax[0,1].plot(field5)
ax[0,2].plot(field6)
ax[1,0].plot(field4<0)
ax[1,1].plot(field5<0)
ax[1,2].plot(field6<0)
plt.show()